In [23]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from langchain.document_loaders import TextLoader

import os
from dotenv import load_dotenv


In [24]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [25]:
# 1. Prepare Documents

loader = TextLoader("data.txt")  # <- replace with your file
docs = loader.load()
# texts = [
#     Document(page_content="In 2024, Tesla launched a new robotaxi service in California."),
#     Document(page_content="Tesla's 2024 update includes Dojo supercomputer improvements."),
#     Document(page_content="In 2023, Tesla's Cybertruck was announced."),
# ]

In [26]:
# 2. Split documents
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = splitter.split_documents(docs)

In [27]:
# 3. Embed and store in vector DB
db = Chroma.from_documents(docs, embedding=OpenAIEmbeddings(), persist_directory="./db")
retriever = db.as_retriever()

In [28]:
# 4. Tagging Chain (optional metadata extraction)
tag_prompt = ChatPromptTemplate.from_template("""
Extract tags like company and year from the following question:

{query}
""")
tag_chain = tag_prompt | ChatOpenAI(temperature=0) | (lambda x: {"metadata": x.content})

In [29]:
# 5. QA Prompt Template
qa_prompt = ChatPromptTemplate.from_template("""
Answer the following question using only the context below.
If the answer is not in the context, say "I don't know."

Context:
{context}

Question: {input}
""")

In [30]:

# 6. Create QA chain (RAG-style)
llm = ChatOpenAI(temperature=0)
qa_chain = qa_prompt | llm

In [31]:
# 7. Full pipeline
def ask_question(query):
    # Step 1: Tag question (optional)
    tags = tag_chain.invoke({"query": query})
    print("Tags:", tags)

    # Step 2: Retrieve relevant documents
    relevant_docs = retriever.get_relevant_documents(query)

    # Step 3: QA over context
    response = qa_chain.invoke({"input": query, "context": relevant_docs})
    print("Full response:", response)

    # Step 4: Safe return
    if isinstance(response, dict):
        return response.get("output") or response.get("answer") or response.get("result") or response
    return response


In [32]:

# 8. Example usage
question = "What all products did tesla launched in 2024?"
print("Answer:", ask_question(question))

Tags: {'metadata': 'company: tesla\nyear: 2024'}
Full response: content="I don't know." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 211, 'total_tokens': 216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bf9YEceQFp13PiMefNyHzKFwMhNHI', 'finish_reason': 'stop', 'logprobs': None} id='run--1c8851f5-7d5e-4553-8d4d-0a747cf68e3a-0' usage_metadata={'input_tokens': 211, 'output_tokens': 5, 'total_tokens': 216, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Answer: content="I don't know." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 211, 'total_tokens': 216, 'complet

Extracted tags: {
  "company": "Tesla",
  "year": "2024"
}
Answer: I don't know.
Final answer: I don't know.
